# 실험적인 코드

TF-IDF 를 직접 구현도 해보고, 모듈도 써본 테스트 코드

완성본은 main 이나 main(최종)를 확인 바람.

직접 구현한 코드는 연산시 메모리 부족 문제로 2만개씩만 계산함! 그래서 대부분 3등분 되어있다.

모듈은 6만개 이상 연산해도 메모리가 널널하다. 

In [1]:
import pandas as pd
import numpy as np
from math import log
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('./pretreatment_data/yeonhap_pre.csv', names=['CNo', 'Subject', 'Contents'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64122 entries, 0 to 64121
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNo       64122 non-null  object
 1   Subject   64122 non-null  object
 2   Contents  64122 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [3]:
et_df = pd.read_csv('./pretreatment_data/etoday_pre.csv', names=['CNo', 'Subject', 'Contents'])
et_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9747 entries, 0 to 9746
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNo       9747 non-null   object
 1   Subject   9747 non-null   object
 2   Contents  9747 non-null   object
dtypes: object(3)
memory usage: 228.6+ KB


## 단어 리스트 뽑아내기

커널을 다시 시작할 때 마다 token_list_ 는 만들어 줘야 한다.

2~3분 밖에 안 걸리니깐 기다리자.

In [ ]:
# 작업 시간 2~3분 정도 걸림
# 모든 문서에서 사용된 단어들을 뽑아내는 작업
temp = df['Contents'][0]
token_list_all = []
cnt = 0
for doc in df['Contents'][1:]:
    temp = temp + " " + doc
    if cnt % 1000 == 0:
        print(cnt, "완료")
    cnt+=1
token_list_all = temp.split(" ")
token_list_all = list(set(token_list_all))
token_list_all = token_list_all[1:]
print(token_list_all)
print(len(token_list_all))

In [ ]:
with open('token_list_all.pickle', 'wb') as handle:
    pickle.dump(token_list_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("idf_dit_1.pickle", 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
# 작업 시간 2~3분 정도 걸림
# 20000개 문서에서 사용된 단어들을 뽑아내는 작업
temp = ""
token_list_1 = []
for doc in df['Contents'][:20000]:
    temp = temp + " " + doc
token_list_1 = temp.split(" ")
token_list_1 = list(set(token_list_1))
token_list_1 = token_list_1[1:]
print(token_list_1)

In [ ]:
temp = ""
token_list_2 = []
for doc in df['Contents'][20000:40000]:
    temp = temp + " " + doc
token_list_2 = temp.split(" ")
token_list_2 = list(set(token_list_2))
token_list_2 = token_list_2[1:]
print(token_list_2)

In [ ]:
temp = ""
token_list_3 = []
for doc in df['Contents'][40000:]:
    temp = temp + " " + doc
token_list_3 = temp.split(" ")
token_list_3 = list(set(token_list_3))
token_list_3 = token_list_3[1:]
print(token_list_3)

## 실행에 필요한 함수들

In [4]:
def tf(term, document):
    # document 에서 term 의 등장 횟수를 count
    return document.count(term)

def idf(term, documents):
    # doc_list 에서 term 이 등장한 문서 수를 count
    
    # 함수가 돌 때마다 초기화
    dfn = 0
    for doc in documents:
        # 각 문서마다 해당 언어가 있는지 확인
        if term in doc:
            dfn = dfn + 1
    return log(len(documents)/(dfn+1))
    
def tfidf(term, docu):
    # tf * idf
    return tf(term, docu)*idf(term)

## 단어 카운트 리스트 만들기

IDF 딕션어리를 만들때 dtm (단어 카운트 리스트) 가 필요.

idf_idt pickle이 있다면 넘어가도 된다.

In [ ]:
dtm_1 = []
cnt = 0
for doc in df['Contents'][:20000]:
    # document term matrix (문서별 단어 등장 횟수) 를 구현해보자, 4 (문서) x 433 (단어) 의 행렬을 리스트로 구성하면 된다

    # 방금 추가한 문서의 리스트
    if cnt % 10 == 0:
        print("{} page 완료".format(cnt))
    dtm_1.append([])
    # docnment = doc.split(" ") -> split 안해도 ㄱㅊ. 안에 있는지 없는지 찾아 낼 수 있음.
    for token in token_list_1:
        # -1은 마지막 인덱스
        dtm_1[-1].append(tf(token, doc))
    cnt += 1

In [ ]:
dtm_2 = []
cnt = 0
for doc in df['Contents'][20000:40000]:
    # document term matrix (문서별 단어 등장 횟수) 를 구현해보자, 4 (문서) x 433 (단어) 의 행렬을 리스트로 구성하면 된다

    # 방금 추가한 문서의 리스트
    if cnt % 10 == 0:
        print("{} page 완료".format(cnt))
    dtm_2.append([])
    for token in token_list_2:
        # -1은 마지막 인덱스
        dtm_2[-1].append(tf(token, doc))
    cnt += 1

In [ ]:
dtm_3 = []
cnt = 0
for doc in df['Contents'][40000:]:
    # document term matrix (문서별 단어 등장 횟수) 를 구현해보자, 4 (문서) x 433 (단어) 의 행렬을 리스트로 구성하면 된다

    # 방금 추가한 문서의 리스트
    if cnt % 10 == 0:
        print("{} page 완료".format(cnt))
    dtm_3.append([])
    for token in token_list_3:
        # -1은 마지막 인덱스
        dtm_3[-1].append(tf(token, doc))
    cnt += 1

pandas로 변환해서 봐도 되는데, 용량이 워낙 큰 바람에 메모리 부족

In [ ]:
dtm_pd = pd.DataFrame(dtm, columns=token_list)
print(dtm_pd)

In [ ]:
print(dtm_pd.iloc[1])

## IDF 딕션어리 만들기

token list 가 필요하다

In [ ]:
# 전부
# 그룹 0
idf_dit_all = {}
cnt = 0
for token in token_list_all:
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    idf_dit_all[token] = idf(token, df['Contents'])
    cnt+=1

In [ ]:
# 2만까지
# 그룹 1
idf_dit_1 = {}
cnt = 0
for token in token_list_1:
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    idf_dit_1[token] = idf(token, df['Contents'][:20000])
    cnt+=1

In [ ]:
# 4만까지
# 그룹 2
idf_dit_2 = {}
cnt = 0
for token in token_list_2:
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    idf_dit_2[token] = idf(token, df['Contents'][20000:40000])
    cnt+=1

In [ ]:
# 끝까지
# 그룹 3
idf_dit_3 = {}
cnt = 0
for token in token_list_3:
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    idf_dit_3[token] = idf(token, df['Contents'][40000:])
    cnt+=1

pickle로 저장

In [ ]:
with open('idf_dit_all.pickle', 'wb') as handle:
    pickle.dump(idf_dit_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(idf_dit_1 == b)

## TF-IDF 리스트 만들기

위에서 만든 idf 딕셔너리가 필요하다

=====>>> 모듈로 대체한다!!!


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df['Contents'])
print(tfidf_matrix.shape)

(64122, 65825)


In [ ]:
# 2만개씩
with open("idf_dit_1.pickle", 'rb') as handle:
    dit_1 = pickle.load(handle)

tfidf_list_1 = []
cnt = 0
for doc in df['Contents'][:20000]:
    tfidf_list_1.append([])
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    for token in token_list_1:
        tfidf_list_1[-1].append(tf(token, doc)*dit_1[token])
    cnt+=1

In [ ]:
# 1만개씩
with open("idf_dit_1.pickle", 'rb') as handle:
    dit_1 = pickle.load(handle)

tfidf_list_1_1 = []
cnt = 0
for doc in df['Contents'][:10000]:
    tfidf_list_1_1.append([])
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    for token in token_list_1:
        tfidf_list_1_1[-1].append(tf(token, doc)*dit_1[token])
    cnt+=1

In [ ]:
# 5000개씩
with open("idf_dit_1.pickle", 'rb') as handle:
    dit_1 = pickle.load(handle)

tfidf_list_1_1_1 = []
cnt = 0
for doc in df['Contents'][:5000]:
    tfidf_list_1_1_1.append([])
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    for token in token_list_1:
        tfidf_list_1_1_1[-1].append(tf(token, doc)*dit_1[token])
    cnt+=1

In [ ]:
# 1000개씩
with open("idf_dit_1.pickle", 'rb') as handle:
    dit_1 = pickle.load(handle)

test_1000 = []
cnt = 0
for doc in df['Contents'][:1000]:
    test_1000.append([])
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    for token in token_list_1:
        test_1000[-1].append(tf(token, doc)*dit_1[token])
    cnt+=1

In [ ]:
with open('tfidf_list_1_1_1.pickle', 'wb') as handle:
    pickle.dump(tfidf_list_1_1_1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("tfidf_list_1_1_1.pickle", 'rb') as handle:
    b = pickle.load(handle)

## 비교할 이투데이 뉴스 idf 만들기

In [38]:
with open("idf_dit_all.pickle", 'rb') as handle:
    dit_1 = pickle.load(handle)  

token_list_all = list(tfidf.vocabulary_.keys())

cut_list = et_df['Contents'][100:150]
vs_list = []
for doc in cut_list:
    vs_list.append([])
    for token in token_list_all:
        vs_list[-1].append(tf(token, doc)*dit_1[token])

In [45]:
type(cut_list)

pandas.core.series.Series

In [39]:
dd = np.array(vs_list)
print(dd.shape)

(50, 65825)


## 코사인 유사도 계산

=======>>> 모듈로 대체!!

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity([vs_list[30][:]], tfidf_matrix)
print(cut_list['Subject'][66])

KeyError: 'Subject'

In [34]:
cosine_matrix.shape

(1, 64122)

In [35]:
a = 0
for i in cosine_matrix[0]:
    if a < i:
        a = i
print(a)

0.09454831194582672


In [36]:
# news title과 id를 맵핑할 dictionary를 생성
news2id = {}
for i, c in enumerate(df['Subject']):
    news2id[i] = c

# id와 news title를 매핑할 dictionary를 생성
id2news = {}
for i, c in news2id.items():
    id2news[c] = i

In [37]:
idx = id2news['첫 정규앨범 블랙핑크 "할 수 있는 음악 다 보여주고 싶었죠"']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx])]
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

IndexError: index 1105 is out of bounds for axis 0 with size 1

In [ ]:
sim_scores[0:5]

In [ ]:
sim_scores = [(news2id[i], score) for i, score in sim_scores[0:5]]
sim_scores

In [ ]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:

from numpy import dot
from numpy.linalg import norm

sim = 0
cnt = 0
for i, tf_idf in enumerate(test_1000):
    sim = cos_sim(vs_list[9], tf_idf)
    if sim >= 0.6:
        print(df['Subject'][i], sim)
        print(et_df['Subject'][i])
    if cnt % 100 == 0:
        print("{} 완료".format(cnt))
    cnt+=1

In [ ]:
import numpy as np
cos_sim(vs_list[5], tf_idf)